Reuters-21578, Distribution 1.0

Comparison of classic text representation methods
(bag-of-words) with vector
representations of words obtained from deep
models (word embeddings) in the task of
categorizing texts.

In [29]:
#importy
import nltk
from nltk.corpus import reuters
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import re



from sklearn.metrics import confusion_matrix, accuracy_score

In [31]:
training_ids = [doc_id for doc_id in reuters.fileids() if doc_id.startswith('training')]

cat_counts = dict()

for id in training_ids:
    for i in range(len(reuters.categories(id))):
        category = reuters.categories(id)[i]
        cat_counts[category] = cat_counts.get(category, 0) + 1

sorted(cat_counts.items(), key=lambda x: x[1], reverse=True)[:30]

categories = [item[0] for item in sorted(cat_counts.items(), key=lambda x: x[1], reverse=True)[:70]]




In [32]:
corpus = {}
tokens = []
DF = {}
term_frequency = []
ids = []
doc_lengths = []

DOCUMENTS = 200

#limited due to memory consumption
training_ids = [doc_id for doc_id in reuters.fileids() if doc_id.startswith('training')][:DOCUMENTS]

for i, id in enumerate(training_ids):
    document = reuters.raw(id)

    vocab = tokenize(document)
    tokens += [vocab]

    for w in vocab:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

    #corpus[f'doc{i}'] = dict((tok, 1) for tok in vocab)
    corpus[id] = dict((tok, 1) for tok in vocab)
    ids.append(id)
    doc_lengths.append(len(vocab))

for i in DF:
    DF[i] = len(DF[i])

for token in vocab:
    term_frequency.append(DF[token])


lexicon = sorted(set(sum(tokens, [])))

In [33]:
training_ids2 = [id for id in training_ids if reuters.categories(id)[0] in categories]

test_ids = [id for id in reuters.fileids() if id.startswith('test') and reuters.categories(id)[0] in categories]

categories = {category: id for id, category in enumerate(reuters.categories())}


In [ ]:
### BOW

sentence_bow = {}
for token in document.split():
    sentence_bow[token] = 1

#sorted(sentence_bow.items())

bag_of_words = Counter(token_sequence)
sorted(bag_of_words.items(), key=lambda x: x[1], reverse=True)
bag_of_words.most_common(4)

In [70]:
train_data = []
test_data = []

for id in training_ids2:
    train_data.append(clean_text(reuters.raw(id)))

for id in test_ids:
    test_data.append(clean_text(reuters.raw(id)))

train_target = [categories[reuters.categories(id)[0]] for id in training_ids2]
test_target = [categories[reuters.categories(id)[0]] for id in test_ids]

In [ ]:
zero_vector = OrderedDict((token, 0) for token in lexicon)

# Vectorization

doc_vectors = []
doc_vectors2 = []

for i, id in enumerate(training_ids):
    document = reuters.raw(id)
    vec = copy.copy(zero_vector)

    vocab = re_pattern.split(document.lower())
    vocab = [token for token in vocab if token not in ',.!?\'\"-\t\n <>;']
    vocab = [token for token in vocab if token not in stop_words]

    token_counts = Counter(vocab)
    for key, value in token_counts.items():
        vec[key] = value #/ len(lexicon)
        doc_vectors.append(vec)

    doc_vectors2.append(list(doc_vectors[i].items()))


In [ ]:
tf_idf = {}

for doc in corpus:
    for token in corpus[doc]:
        tf = corpus[doc][token] / len(corpus[doc])
        df = token_counts[token] / N
        idf = np.log(N / (df + 1))  #add 1 to avoid possibility of dividing by zero if there's no instance of the word in lexicon
        tf_idf[doc, token] = tf*idf

#tf_idf['doc0', 'review']

In [71]:
# Build the model with naive bayes
model_tfidf = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_count = make_pipeline(CountVectorizer(), MultinomialNB())
# Train the model using the training data
model_tfidf.fit(train_data, train_target)
# Predict the categories of the test data
predicted_categories = model_tfidf.predict(test_data)

In [ ]:
def matching_score(k, query):
    preprocessed_query = tokenize(query)
    tokens = tokenize(str(preprocessed_query))
    
    query_weights = {}

    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)
    
    l = []
    
    for i in query_weights[:k]:
        l.append(i[0])
    
    print(l)

In [ ]:
matching_score2(5, reuters.raw('training/9865'))
reuters.categories('training/9865')
reuters.categories('training/10120')


Ranking by cosine similarity

In [ ]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [ ]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [ ]:
def gen_vector(tokens):

    Q = np.zeros((len(lexicon)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))

        try:
            ind = lexicon.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [ ]:
def cosine_similarity(k, query):
    preprocessed_query = tokenize(query)
    tokens = tokenize(str(preprocessed_query))
       
    d_cosines = []

    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        
    out = np.array(d_cosines).argsort()[-k:][::-1]

    #for i in out:
        #print(i, dataset[i][0])
    return out

In [72]:
mat = confusion_matrix(test_target, predicted_categories)


In [69]:
print("The accuracy is {}".format(accuracy_score(test_target, predicted_categories)))

The accuracy is 0.6205059920106525


In [74]:
predicted = sum([test_target[i] == predicted_categories[i] for i in range(len(test_target))])

all = len(test_target)

predicted/all

0.6344873501997337

In [39]:
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Phrases
import tensorflow as tf

def tokenize(doc):
    re_pattern = re.compile(r'([\s\d.,!?;\"\'-+])+')
    #nltk.download('stopwords') 
    stop_words = nltk.corpus.stopwords.words('english')
    stop_words.append('&lt')

    tokens = re_pattern.split(doc.lower())
    tokens = [token for token in tokens if token not in ',.!?\'\"-\t\n ;']
    tokens = [token for token in tokens if token not in stop_words]

    return tokens

def clean_text(
    string: str, 
    punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
    stop_words=['the', 'a', 'and', 'is', 'be', 'will']) -> str:
    """
    A method to clean text 
    """

    # Removing the punctuations
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 

    # Converting the text to lower
    string = string.lower()

    # Removing stop words
    string = ' '.join([word for word in string.split() if word not in stop_words])

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()

    stemmer = PorterStemmer()
    ' '.join([stemmer.stem(w).strip("'") for w in token_sequence])

    return string   

In [46]:
corpus = []

for id in training_ids:
    corpus.append(tokenize(reuters.raw(id)))

In [42]:
model1 = gensim.models.Word2Vec(corpus, min_count = 1, window = 5)